<h1>DeviceEngine Class

Dedicated engine for device data, inherited from Core Engine. Each DeviceEngine class object will represent a unique device with its own set of processing parameters and results.

In [1]:
from src.StreamPort.device.DeviceEngine import DeviceEngine
from src.StreamPort.core.ProcessingSettings import ProcessingSettings
from src.StreamPort.core.ProjectHeaders import ProjectHeaders

In [2]:
settings = ProcessingSettings()

In [3]:
#specify path to get analyses from
base_dir = r'C:\Users\PC0118\Desktop\ExtractedSignals'

In [4]:

#Creates an empty DeviceEngine object and prints it
dev = DeviceEngine(source = base_dir)
dev.print()


DeviceEngine 
  name: None 
  author: None 
  path: None 
  date: 2024-05-25 18:00:02.439063 
  analyses: 0 
  settings: 0 



In [5]:
dev.add_headers(headers = {'name': 'Pressure Curve Analysis', 'author': 'Sandeep H.'})
dev.print()


DeviceEngine 
  name: Pressure Curve Analysis 
  author: Sandeep H. 
  path: None 
  date: 2024-05-25 18:00:02.439063 
  analyses: 0 
  settings: 0 



In [6]:
dev.add_settings(settings)
dev.print()


DeviceEngine 
  name: Pressure Curve Analysis 
  author: Sandeep H. 
  path: None 
  date: 2024-05-25 18:00:02.439063 
  analyses: 0 
  settings: 1 



<h1>DeviceAnalysis Class

Each DeviceAnalysis object is a child of the Analysis Class. It holds the details of an Analysis or list of Analyses for each individual device.

In [7]:
from src.StreamPort.device.DeviceAnalysis import DeviceAnalysis

#Creates an empty DeviceAnalysis object and prints it
devAnalysis = DeviceAnalysis()
devAnalysis.print()


Analysis
  name: None
  replicate: None
  blank: None
  data:
  Empty



Read analysis objects from engine. DeviceEngine's read_device_spectra() returns a DeviceAnalysis Object or a list of DeviceAnalysis objects, besides printing the dataframes for each unique Method, paired with the metadata(Date, Runtime) for each curve.

In [8]:
analyses = dev.find_analyses()

Sample - 02-blank : 
start date : 15:11:06 01/31/24
end date : 15:16:50 01/31/24
runtime : 0:05:44

Sample - 240131_Irino_Kali--003 : 
start date : 15:17:00 01/31/24
end date : 15:22:43 01/31/24
runtime : 0:05:43

Sample - 240131_Irino_Kali--004 : 
start date : 15:22:54 01/31/24
end date : 15:28:38 01/31/24
runtime : 0:05:44

Sample - 240131_Irino_Kali--005 : 
start date : 15:28:48 01/31/24
end date : 15:34:31 01/31/24
runtime : 0:05:43

Sample - 240131_Irino_Kali--006 : 
start date : 15:34:41 01/31/24
end date : 15:40:25 01/31/24
runtime : 0:05:44

Sample - 240131_Irino_Kali--007 : 
start date : 15:40:35 01/31/24
end date : 15:46:19 01/31/24
runtime : 0:05:44

Sample - 240131_Irino_Kali--008 : 
start date : 15:46:29 01/31/24
end date : 15:52:14 01/31/24
runtime : 0:05:45

Sample - 240131_Irino_Kali--009 : 
start date : 15:52:24 01/31/24
end date : 15:58:08 01/31/24
runtime : 0:05:44

Sample - 240131_Irino_Kali--010 : 
start date : 15:58:18 01/31/24
end date : 16:04:00 01/31/24
runtime

In [9]:
for ana in analyses:
    print("\n")
    ana.validate()
    print("Analysis Object : \n")
    print(f"Analysis : {ana.print()}")
    print("Data of Analysis : \n")
    print(ana.data)
    print("\n")



Analysis Object : 


Analysis
  name: Analysis - Onivyde_240129_5 15:11:06 01/31/24
  replicate: None
  blank: None
  data:
    Device Pressure Analysis - 15:11:06 01/31/24 (size 5)

Analysis : None
Data of Analysis : 

{'Device Pressure Analysis - 15:11:06 01/31/24': array(['Method : Onivyde_240129_5', 'Sample : Sample - 02-blank',
       'Start date : 15:11:06 01/31/24', 'Runtime : 0:05:44',
       'Time since last flush : NA'], dtype='<U30')}




Analysis Object : 


Analysis
  name: Analysis - Onivyde_240129_5 15:17:00 01/31/24
  replicate: None
  blank: None
  data:
    Device Pressure Analysis - 15:17:00 01/31/24 (size 5)

Analysis : None
Data of Analysis : 

{'Device Pressure Analysis - 15:17:00 01/31/24': array(['Method : Onivyde_240129_5',
       'Sample : Sample - 240131_Irino_Kali--003',
       'Start date : 15:17:00 01/31/24', 'Runtime : 0:05:43',
       'Time since last flush : NA'], dtype='<U40')}




Analysis Object : 


Analysis
  name: Analysis - Onivyde_240129_5 15:

In [10]:
dev.add_analyses(analyses)

In [11]:
dev.print()


DeviceEngine 
  name: Pressure Curve Analysis 
  author: Sandeep H. 
  path: None 
  date: 2024-05-25 18:00:02.439063 
  analyses: 141 
  settings: 1 



In [12]:
devAnalysis.print()


Analysis
  name: None
  replicate: None
  blank: None
  data:
  Empty



In [13]:
dev.add_analyses(devAnalysis)
dev.print()


DeviceEngine 
  name: Pressure Curve Analysis 
  author: Sandeep H. 
  path: None 
  date: 2024-05-25 18:00:02.439063 
  analyses: 142 
  settings: 1 



In [14]:
dev.remove_analyses(0)
dev.print()


DeviceEngine 
  name: Pressure Curve Analysis 
  author: Sandeep H. 
  path: None 
  date: 2024-05-25 18:00:02.439063 
  analyses: 141 
  settings: 1 

